In [15]:
# Dataset: BBC news
%run -i "../util/lang_utils.ipynb"

In [16]:
from datasets import load_dataset
from nltk import word_tokenize
from math import ceil
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords


In [17]:
train_dataset = load_dataset("SetFit/bbc-news", split="train")
test_dataset = load_dataset("SetFit/bbc-news", split="test")
train_df = train_dataset.to_pandas()
test_df = test_dataset.to_pandas()
print(train_df)
print(test_df)

Repo card metadata block was not found. Setting CardData to empty.
Repo card metadata block was not found. Setting CardData to empty.


                                                   text  label     label_text
0     wales want rugby league training wales could f...      2          sport
1     china aviation seeks rescue deal scandal-hit j...      1       business
2     rock band u2 break ticket record u2 have smash...      3  entertainment
3     markets signal brazilian recovery the brazilia...      1       business
4     tough rules for ringtone sellers firms that fl...      0           tech
...                                                 ...    ...            ...
1220  us economy shows solid gdp growth the us econo...      1       business
1221  microsoft releases bumper patches microsoft ha...      0           tech
1222  stuart joins norwich from addicks norwich have...      2          sport
1223  why few targets are better than many the econo...      1       business
1224  boothroyd calls for lords speaker betty boothr...      4       politics

[1225 rows x 3 columns]
                                       

In [14]:
train_dataset = load_dataset("multi_news", split="train")
test_dataset = load_dataset("multi_news", split="test")
train_df = train_dataset.to_pandas()
test_df = test_dataset.to_pandas()
print(train_df)

                                                document  \
0      National Archives \n \n Yes, it’s that time ag...   
1      LOS ANGELES (AP) — In her first interview sinc...   
2      GAITHERSBURG, Md. (AP) — A small, private jet ...   
3      Tucker Carlson Exposes His Own Sexism on Twitt...   
4      A man accused of removing another man's testic...   
...                                                  ...   
44967  More than 670,000 copies of the Pearls’ self-p...   
44968  Seeking out cost-conscious consumers who have ...   
44969  Click to email this to a friend (Opens in new ...   
44970  BARRINGTON, R.I. (AP) — Women clad in yoga pan...   
44971  Based on a ‘real’ story, the hit John Travolta...   

                                                 summary  
0      – The unemployment rate dropped to 8.2% last m...  
1      – Shelly Sterling plans "eventually" to divorc...  
2      – A twin-engine Embraer jet that the FAA descr...  
3      – Tucker Carlson is in deep doodoo w

In [18]:
vectorizer = TfidfVectorizer(stop_words='english', min_df=2, max_df=0.95)
vectorizer.fit(train_df["text"])

TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')

In [19]:
def sort_data_tfidf_score(coord_matrix):
    tuples = zip(coord_matrix.col, coord_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)    

In [20]:
def get_keyword_strings(vectorizer, num_words, sorted_vector):
    words = []
    index_dict = vectorizer.get_feature_names_out()
    for (item_index, score) in sorted_vector[0:num_words]:
        word = index_dict[item_index]
        words.append(word)
    return words

In [21]:
def get_keywords_simple(vectorizer, input_text, num_output_words=10):
    vector = vectorizer.transform([input_text])
    sorted = sort_data_tfidf_score(vector.tocoo())
    words = get_keyword_strings(vectorizer, num_output_words, sorted)
    return words

In [22]:
print(test_df.iloc[0]["text"])
keywords = get_keywords_simple(vectorizer, test_df.iloc[0]["text"])
print(keywords)

carry on star patsy rowlands dies actress patsy rowlands  known to millions for her roles in the carry on films  has died at the age of 71.  rowlands starred in nine of the popular carry on films  alongside fellow regulars sid james  kenneth williams and barbara windsor. she also carved out a successful television career  appearing for many years in itv s well-loved comedy bless this house. rowlands died in hove on saturday morning  her agent said.  born in january 1934  rowlands won a scholarship to the guildhall school of speech and drama scholarship when she was just 15.  after spending several years at the players theatre in london  she made her film debut in 1963 in tom jones  directed by tony richardson. she made her first carry on film in 1969 where she appeared in carry on again doctor. rowlands played the hard-done-by wife or the put-upon employee as a regular carry on star. she also appeared in carry on at your convenience  carry on matron and carry on loving  as well as othe

In [23]:
stop_words = list(stopwords.words('english'))
stop_words.remove("the")
trigram_vectorizer = TfidfVectorizer(stop_words=stop_words, min_df=2, ngram_range=(1,3), max_df=0.95)
trigram_vectorizer.fit(train_df["text"])

TfidfVectorizer(max_df=0.95, min_df=2, ngram_range=(1, 3),
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...])

In [24]:
def get_keyword_strings_all(vectorizer, sorted_vector):
    words = []
    index_dict = vectorizer.get_feature_names_out()
    for (item_index, score) in sorted_vector:
        word = index_dict[item_index]
        words.append(word)
    return words

In [30]:
keywords = get_keywords_simple(vectorizer, test_df.iloc[0]["text"])
print(keywords)

['carry', 'theatre', 'scholarship', 'appeared', 'films', 'mrs', 'agent', 'drama', 'died', 'school']


In [25]:
def get_keywords_complex(vectorizer, input_text, spacy_model, num_words=70):
    keywords = []
    doc = spacy_model(input_text)
    vector = vectorizer.transform([input_text])
    sorted = sort_coo(vector.tocoo())
    ngrams = get_keyword_strings_all(vectorizer, sorted)
    ents = [ent.text.lower() for ent in doc.noun_chunks]
    for i in range(0, num_words):
        keyword = ngrams[i]
        if keyword.lower() in ents and not keyword.isdigit() and keyword not in keywords:
            keywords.append(keyword)
    return keywords

In [29]:
keywords = get_keywords_complex(trigram_vectorizer, test_df.iloc[0]["text"], small_model)
print(keywords)

['carry', 'films', 'stage', 'several years', 'saturday morning', 'star', 'film', 'london', 'beauty', 'the good', 'many years', 'directors']
